<font size="8"> **Adding environmental data from ACCESS-OM2-01 to unique background points** </font>  
In this notebook, we will extract environmental data from the ACCESS-OM2-01 model outputs and add it to our data frame containing unique crabeater sightings per month and grid cell (see `04b_Creating_background_masks.ipynb` for more information.

# Setting working directory
In order to ensure these notebooks work correctly, we will set the working directory. We assume that you have saved a copy of this repository in your home directory (represented by `~` in the code chunk below). If you have saved this repository elsewhere in your machine, you need to ensure you update this line with the correct filepath where you saved these notebooks.

In [1]:
import os
os.chdir(os.path.expanduser('~/Chapter2_Crabeaters/Scripts'))

# Loading other relevant libraries

In [2]:
from dask.distributed import Client
from glob import glob
#Accessing model data
import cosima_cookbook as cc
#Useful functions
import UsefulFunctions as uf
#Dealing with data
import xarray as xr
import pandas as pd
import numpy as np
#Data visualisation
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# Paralellising work 

In [3]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 4
Total threads: 12,Total memory: 48.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43687,Workers: 4
Dashboard: /proxy/8787/status,Total threads: 12
Started: Just now,Total memory: 48.00 GiB
Comm: tcp://127.0.0.1:45645,Total threads: 3
Dashboard: /proxy/33373/status,Memory: 12.00 GiB
Nanny: tcp://127.0.0.1:46023,


# Loading unique crabeater seal observations data frame

In [103]:
#Loading dataset as pandas data frame
crabeaters = pd.read_csv('../Cleaned_Data/unique_background_20x_obs_grid.csv')

#Ensuring date column is formatted correctly (year-month)
crabeaters['date'] = crabeaters.apply(lambda x: f'{x.year}-{str(x.month).zfill(2)}', axis = 1)

#Checking results
crabeaters

,date,year,month,yt_ocean,xt_ocean,yu_ocean,xu_ocean,season_year,life_stage,decade,sector,zone,presence
0,1990-12,1990,12,-63.046,97.05,-63.068,97.0,summer,weaning,1990,Central Indian,Antarctic,0
1,1985-12,1985,12,-63.629,81.65,-63.651,81.6,summer,weaning,1980,Central Indian,Antarctic,0
2,1984-11,1984,11,-63.540,81.15,-63.562,81.1,autumn,weaning,1980,Central Indian,Antarctic,0
3,1987-11,1987,11,-65.058,129.35,-65.037,129.3,autumn,weaning,1980,Central Indian,Antarctic,0
4,1999-12,1999,12,-62.358,103.65,-62.335,103.6,summer,weaning,1990,Central Indian,Antarctic,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10774,2001-11,2001,11,-63.850,121.05,-63.828,121.0,autumn,weaning,2000,Central Indian,Antarctic,0
10775,1998-11,1998,11,-65.269,119.85,-65.248,119.8,autumn,weaning,1990,Central Indian,Antarctic,0
10776,1996-11,1996,11,-64.931,134.25,-64.952,134.2,autumn,weaning,1990,Central Indian,Antarctic,0
10777,1999-12,1999,12,-64.244,123.05,-64.266,123.0,summer,weaning,1990,Central Indian,Antarctic,0


# Loading crabeater seal masks

In [295]:
mask_all = xr.open_dataarray('/g/data/v45/la6889/Chapter2_Crabeaters/mask_background_20x_obs_ocean_grid.nc')
mask_vel = xr.open_dataarray('/g/data/v45/la6889/Chapter2_Crabeaters/mask_background_20x_obs_vel_grid.nc')

# Adding values for static variables only
Static variables referred to any physical variables that do not change over time (at least not during the time period of our interest). Examples include depth of the water column and distance to coastline. Given that we only have one value for these variables, the process of extracting data is relatively simple. We do not need to take into account the date observations were collected.

## Defining dictionary with information about static variables
This dictionary contains the column labels for each and the name of the files for each static variable to be included in our analysis. We will also define a variable containing the full path to the folder where all static variables are stored.

In [107]:
#Full path to static variables
base_dir_static = '/g/data/v45/la6889/Chapter2_Crabeaters/Static_Variables/'

#List of static variables
varDict = {'bottom_slope_deg': 'bathy_slope_GEBCO_2D.nc',
           'dist_shelf_km': 'distance_shelf.nc',
           'dist_coast_km': 'distance_coastline.nc',
           'depth_m': 'bathy_GEBCO_2D.nc'}

## Extracting data for each observation and adding it to a new column in crabeater data

In [45]:
#Looping through dictionary keys
for var in varDict:
    #Creating full path to file of interest
    file_path = os.path.join(base_dir_static, varDict[var])
    #Load as raster
    ras = xr.open_dataarray(file_path).sel(yt_ocean = slice(-80, -45))
    ras.name = var
    #Applying mask
    ras_masked = ras.where(mask_all == 0)
    #Transforming masked array into data frame
    ras_df = ras_masked.to_series().dropna().reset_index()
    #Rounding up coordinate values
    ras_df = ras_df.round({'yt_ocean': 3, 'xt_ocean': 3})
    #Renaming masked data before merging to observations
    ras_df.rename(columns = {0: var}, inplace = True)
    #Adding to crabeater observations data frame
    crabeaters = crabeaters.merge(ras_df, on = ['yt_ocean', 'xt_ocean'], how = 'left').sort_values(['yt_ocean', 'xt_ocean'])
    
#Checking results
crabeaters

,date,year,month,yt_ocean,xt_ocean,yu_ocean,xu_ocean,season_year,life_stage,decade,sector,zone,presence,bottom_slope_deg,dist_shelf_km,dist_coast_km,depth_m
0,1999-12,1999,12,-69.239,75.05,-69.260,75.1,summer,weaning,1990,East Indian,Antarctic,0,89.812088,504.080212,140.255558,777.866638
1,1999-12,1999,12,-69.239,75.85,-69.260,75.9,summer,weaning,1990,Central Indian,Antarctic,0,89.953110,506.158223,143.956368,503.000000
2,1999-12,1999,12,-69.239,76.05,-69.260,76.1,summer,weaning,1990,East Indian,Antarctic,0,89.948738,507.046161,145.996809,522.900024
3,1999-12,1999,12,-69.239,77.25,-69.260,77.3,summer,weaning,1990,Central Indian,Antarctic,0,NaN,509.340574,166.208399,NaN
4,1999-12,1999,12,-69.155,73.95,-69.134,74.0,summer,weaning,1990,Central Indian,Antarctic,0,NaN,494.994640,130.924977,599.817383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35561,1999-12,1999,12,-59.442,69.25,-59.468,69.3,summer,weaning,1990,Central Indian,Antarctic,0,89.832825,556.221299,688.617402,4624.497070
35562,1999-12,1999,12,-59.442,71.95,-59.468,72.0,summer,weaning,1990,East Indian,Antarctic,0,89.652817,536.647811,699.380306,4429.308105
35563,1987-11,1987,11,-59.340,70.85,-59.366,70.9,autumn,weaning,1980,Central Indian,Antarctic,0,89.979530,551.535259,703.061410,4767.583496
35564,1998-11,1998,11,-59.340,71.55,-59.366,71.6,autumn,weaning,1990,Central Indian,Antarctic,0,89.929642,548.570655,707.329724,4608.486328


## Saving data frame with static variables
Given that the dynamic variables take some time to extract. We will save intermediary results to avoid having to extract them again.

In [46]:
crabeaters.to_csv('../Cleaned_Data/unique_background_20x_obs_static_env.csv', index = False)

# Adding values for dynamic variables
Given the amount of crabeater seal observations and the time period covered by this dataset, the extraction of these values may take some time. It is recommended to save the data frame after every time a new variable is extracted. This way we can avoid losing data.

In [296]:
crabeaters = pd.read_csv('../Cleaned_Data/unique_background_20x_obs_static_env.csv')
#Ensuring date column is formatted correctly (year-month)
crabeaters['date'] = crabeaters.apply(lambda x: f'{x.year}-{str(x.month).zfill(2)}', axis = 1)
crabeaters

,date,year,month,yt_ocean,xt_ocean,yu_ocean,xu_ocean,season_year,life_stage,decade,sector,zone,presence,bottom_slope_deg,dist_shelf_km,dist_coast_km,depth_m
0,1999-12,1999,12,-69.239,75.05,-69.260,75.1,summer,weaning,1990,East Indian,Antarctic,0,89.812088,504.080212,140.255558,777.86664
1,1999-12,1999,12,-69.239,75.85,-69.260,75.9,summer,weaning,1990,Central Indian,Antarctic,0,89.953110,506.158223,143.956368,503.00000
2,1999-12,1999,12,-69.239,76.05,-69.260,76.1,summer,weaning,1990,East Indian,Antarctic,0,89.948738,507.046161,145.996809,522.90000
3,1999-12,1999,12,-69.239,77.25,-69.260,77.3,summer,weaning,1990,Central Indian,Antarctic,0,NaN,509.340574,166.208399,NaN
4,1999-12,1999,12,-69.155,73.95,-69.134,74.0,summer,weaning,1990,Central Indian,Antarctic,0,NaN,494.994640,130.924977,599.81740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35561,1999-12,1999,12,-59.442,69.25,-59.468,69.3,summer,weaning,1990,Central Indian,Antarctic,0,89.832825,556.221299,688.617402,4624.49700
35562,1999-12,1999,12,-59.442,71.95,-59.468,72.0,summer,weaning,1990,East Indian,Antarctic,0,89.652817,536.647811,699.380306,4429.30800
35563,1987-11,1987,11,-59.340,70.85,-59.366,70.9,autumn,weaning,1980,Central Indian,Antarctic,0,89.979530,551.535259,703.061410,4767.58350
35564,1998-11,1998,11,-59.340,71.55,-59.366,71.6,autumn,weaning,1990,Central Indian,Antarctic,0,89.929642,548.570655,707.329724,4608.48630


## Accessing ACCESS-OM2-01 model outputs
We will create a new `cosima cookbook` session to load the model outputs of interest, and we will also create a dictionary that contains useful information related to data extraction.

In [5]:
#Creating new COSIMA cookbook session
session = cc.database.create_session()

#Creating dictionary with useful information
varDict = {'model': 'ACCESS-OM2-01',
           #ACCESS-OM2-01 cycle 4 (1958-2018)
           'exp': '01deg_jra55v140_iaf_cycle4',
           #ACCESS-OM2-01 cycle 4 extension (2018-2022)
           'exp_ext': '01deg_jra55v140_iaf_cycle4_jra55v150_extension',
           #Temporal resolution
           'freq': '1 monthly',
           #Output folder
           'base_out': '../Cleaned_Data'}

## Loading data frame with ACCESS-OM2-01 outputs
We can use this data frame to find the variable names for the environmental factors that we know are influential for the distribution of crabeater seals.

In [102]:
#Loading data frame with model outputs
# var_acc = cc.querying.get_variables(session, experiment = varDict['exp_ext'], frequency = '1 monthly')

#Searching data frame for variables of interest
var_acc[var_acc.name.str.contains('salt')]

,name,long_name,units,frequency,ncfile,cell_methods,# ncfiles,time_start,time_end
58,fsalt_ai_m,salt flux ice to ocean,kg/m^2/s,1 monthly,output1008/ice/OUTPUT/iceh.2023-03.nc,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00
59,fsalt_m,salt flux ice to ocn (cpl),kg/m^2/s,1 monthly,output1008/ice/OUTPUT/iceh.2023-03.nc,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00
96,salt,Practical Salinity,psu,1 monthly,output1008/ocean/ocean-3d-salt-1-monthly-mean-...,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00
97,salt_xflux_adv,rho*dzt*dyt*u*tracer,kg/sec,1 monthly,output1008/ocean/ocean-3d-salt_xflux_adv-1-mon...,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00
98,salt_yflux_adv,rho*dzt*dxt*v*tracer,kg/sec,1 monthly,output1008/ocean/ocean-3d-salt_yflux_adv-1-mon...,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00
105,sfc_salt_flux_coupler,sfc_salt_flux_coupler: flux from the coupler,kg/(m^2*sec),1 monthly,output1008/ocean/ocean-2d-sfc_salt_flux_couple...,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00
106,sfc_salt_flux_ice,sfc_salt_flux_ice,kg/(m^2*sec),1 monthly,output1008/ocean/ocean-2d-sfc_salt_flux_ice-1-...,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00
107,sfc_salt_flux_restore,sfc_salt_flux_restore: flux from restoring term,kg/(m^2*sec),1 monthly,output1008/ocean/ocean-2d-sfc_salt_flux_restor...,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00
120,surface_salt,Practical Salinity,psu,1 monthly,output1008/ocean/ocean-2d-surface_salt-1-month...,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00


## Completing dictionary with useful variables
Now that we identified the correct name for the variable of our interest, we can complete our dictionary.

In [411]:
#Variable name in the model
varDict['var_mod'] = 'dist_km'
#Name of column where we will store the extracted data
varDict['var_short_name'] = 'dist_ice_edge_km'
#Defining if this variable is related to sea ice or not
varDict['ice_data'] = False
#Checking final dictionary
varDict

{'model': 'ACCESS-OM2-01',
 'exp': '01deg_jra55v140_iaf_cycle4',
 'exp_ext': '01deg_jra55v140_iaf_cycle4_jra55v150_extension',
 'freq': '1 monthly',
 'base_out': '../Cleaned_Data',
 'var_mod': 'dist_km',
 'var_short_name': 'dist_ice_edge_km',
 'ice_data': False}

## Cropping mask to cover the Indian sectors only

In [298]:
mask_all = mask_all.sel(yt_ocean = slice(-80, -40), xt_ocean = slice(30, 170))
mask_vel = mask_vel.sel(yu_ocean = slice(-80, -40), xu_ocean = slice(30, 170))

## Loading data from ACCESS-OM2-01
The crabeater data has data between 1978 and 2022. The ACCESS-OM2-01 originally had a temporal range of 60 years (1958-2018). However, for the 4th model run (used in these notebooks), the model was extended to December 2022. These outputs are available in two different experiments, which are loaded and merged below before data extraction occurs.

In [393]:
#Loading data from fourth cycle (temporal range 1958 to 2018)
var_df = uf.getACCESSdata_SO(varDict['var_mod'], '1978-01', '2019-01', 
                              freq = varDict['freq'], ses = session, minlat = -80, maxlat = -40,
                              exp = varDict['exp'], ice_data = varDict['ice_data'])
#Loading data from fourth cycle extension (2019 to 2022)
var_df_ext = uf.getACCESSdata_SO(varDict['var_mod'], '2019-01', '2023-01', 
                              freq = varDict['freq'], ses = session, minlat = -80, maxlat = -40,
                              exp = varDict['exp_ext'], ice_data = varDict['ice_data'])

#Concatenating both data arrays into one
var_df = xr.concat([var_df, var_df_ext], dim = 'time')

#Transforming longitudes so their range is +/-180 degrees
var_df = uf.corrlong(var_df)

if var_df.name in ['u', 'v']:
    var_df = var_df.sel(xu_ocean = slice(30, 170))
else:
    var_df = var_df.sel(xt_ocean = slice(30, 170))

#Removing duplicate variable
del var_df_ext

## *Optional: Data transformation*
Some variables, such as sea surface temperature are in units of Kelvin instead of $^{\circ}C$ (shown below). In this section, we transform variables to the units of our choice.

In [334]:
#Temperature data is transformed from Kelvins to C
var_df = var_df-273.15

## *Optional: Subsetting surface layer data*
For some ocean variables, we need to subset data to extract surface values or bottom values. Subsetting data for the surface layer is an easy process, we simply need to select the first depth bin available. The `st_ocean` dimension contains the depth bins.

In [385]:
#Selecting the first depth available in the model (i.e. surface layer)
var_df = var_df.isel(st_ocean = 0)

#Checking results - dataset has three dimensions instead of the original four
var_df

<xarray.DataArray 'u' (time: 541, yu_ocean: 781, xu_ocean: 1399)>
dask.array<getitem, shape=(541, 781, 1399), dtype=float32, chunksize=(1, 135, 180), chunktype=numpy.ndarray>
Coordinates:
  * xu_ocean  (xu_ocean) float64 30.1 30.2 30.3 30.4 ... 169.6 169.7 169.8 169.9
  * yu_ocean  (yu_ocean) float64 -79.99 -79.95 -79.9 ... -40.22 -40.14 -40.07
    st_ocean  float64 0.5413
  * time      (time) datetime64[ns] 1977-12-16T12:00:00 ... 2022-12-16T12:00:00
Attributes: (12/13)
    long_name:      i-current
    units:          m/sec
    valid_range:    [-10.  10.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_c geolat_c
    ...             ...
    ncfiles:        ['/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_...
    contact:        Andrew Kiss
    email:          andrew.kiss@anu.edu.au
    created:        2022-04-27
    description:    0.1 degree ACCESS-OM2 global model configuration under in...
    notes:          Run configuration and history: https://github.com/COSIMA/...

## *Optional: Subsetting bottom data*
Subsetting data for the bottom layer is not as straightforward as for the surface. This is because the bathymetry is not the same across the Southern Ocean. To identify the correct depth bin that contains the deepest values for a particular grid cell.

In [394]:
mask_2d = xr.where(~np.isnan(var_df.isel(time = 0)), 1, np.nan)
mask_2d = mask_2d.cumsum('st_ocean').where(~np.isnan(var_df.isel(time = 0)))
mask_2d = xr.where(mask_2d == mask_2d.max('st_ocean'), 1, np.nan)
mask_2d

<xarray.DataArray 'u' (st_ocean: 75, yu_ocean: 781, xu_ocean: 1399)>
dask.array<where, shape=(75, 781, 1399), dtype=float64, chunksize=(19, 135, 180), chunktype=numpy.ndarray>
Coordinates:
  * xu_ocean  (xu_ocean) float64 30.1 30.2 30.3 30.4 ... 169.6 169.7 169.8 169.9
  * yu_ocean  (yu_ocean) float64 -79.99 -79.95 -79.9 ... -40.22 -40.14 -40.07
  * st_ocean  (st_ocean) float64 0.5413 1.681 2.94 ... 5.511e+03 5.709e+03
    time      datetime64[ns] 1977-12-16T12:00:00

In [395]:
var_2d = (mask_2d*var_df).sum('st_ocean')
var_2d

<xarray.DataArray 'u' (yu_ocean: 781, xu_ocean: 1399, time: 541)>
dask.array<sum-aggregate, shape=(781, 1399, 541), dtype=float64, chunksize=(135, 180, 1), chunktype=numpy.ndarray>
Coordinates:
  * xu_ocean  (xu_ocean) float64 30.1 30.2 30.3 30.4 ... 169.6 169.7 169.8 169.9
  * yu_ocean  (yu_ocean) float64 -79.99 -79.95 -79.9 ... -40.22 -40.14 -40.07
  * time      (time) datetime64[ns] 1977-12-16T12:00:00 ... 2022-12-16T12:00:00

In [396]:
if var_df.name in ['u', 'v']:
    var_df = var_2d.transpose('time', 'yu_ocean', 'xu_ocean')
else:
    var_df = var_2d.transpose('time', 'yt_ocean', 'xt_ocean')
#Checking results
var_df

<xarray.DataArray 'u' (time: 541, yu_ocean: 781, xu_ocean: 1399)>
dask.array<transpose, shape=(541, 781, 1399), dtype=float64, chunksize=(1, 135, 180), chunktype=numpy.ndarray>
Coordinates:
  * xu_ocean  (xu_ocean) float64 30.1 30.2 30.3 30.4 ... 169.6 169.7 169.8 169.9
  * yu_ocean  (yu_ocean) float64 -79.99 -79.95 -79.9 ... -40.22 -40.14 -40.07
  * time      (time) datetime64[ns] 1977-12-16T12:00:00 ... 2022-12-16T12:00:00

## Loading dynamic variables derived from calculations
These variables are not directly available in ACCESS-OM2-01, but they have been calculated from model outputs. Refer to folder `02_Environmental_Data` to see the full details of each calculation.

In [412]:
var_df = xr.open_mfdataset(sorted(glob('/g/data/v45/la6889/Chapter2_Crabeaters/SeaIce/Distance_Edge/*.nc')))[varDict['var_mod']]
var_df

<xarray.DataArray 'dist_km' (time: 541, yt_ocean: 713, xt_ocean: 3600)>
dask.array<concatenate, shape=(541, 713, 3600), dtype=float64, chunksize=(1, 713, 3600), chunktype=numpy.ndarray>
Coordinates:
  * time      (time) datetime64[ns] 1977-12-31T12:00:00 ... 2022-12-31T12:00:00
  * yt_ocean  (yt_ocean) float64 -79.97 -79.93 -79.88 ... -45.18 -45.11 -45.04
  * xt_ocean  (xt_ocean) float64 -179.9 -179.8 -179.7 ... 179.8 179.9 180.0
Attributes:
    units:      km
    long_name:  distance to nearest neighbour

## Subsetting variables by time to match observations
By the subsetting the original dataset, we will reduce computing time.

In [413]:
#Creating a list with the year and month when crabeaters were observed - Based on mask time steps
timestep = sorted(crabeaters.date.unique())

#Creating an empty list to keep the subset model data
var_df_int = []
#Looping through each time
for t in timestep:
    var_df_int.append(var_df.sel(time = t))
    
#Creating a new data array with the time steps of interest
var_df_int = xr.concat(var_df_int, dim = 'time')
#Dealing original model data
del var_df

#Checking results - Time steps now match mask
var_df_int

<xarray.DataArray 'dist_km' (time: 25, yt_ocean: 713, xt_ocean: 3600)>
dask.array<concatenate, shape=(25, 713, 3600), dtype=float64, chunksize=(1, 713, 3600), chunktype=numpy.ndarray>
Coordinates:
  * time      (time) datetime64[ns] 1981-12-31T12:00:00 ... 2013-12-31T12:00:00
  * yt_ocean  (yt_ocean) float64 -79.97 -79.93 -79.88 ... -45.18 -45.11 -45.04
  * xt_ocean  (xt_ocean) float64 -179.9 -179.8 -179.7 ... 179.8 179.9 180.0
Attributes:
    units:      km
    long_name:  distance to nearest neighbour

## Applying crabeater observations mask

In [414]:
#Identifying correct mask to apply
if 'xt_ocean' in var_df_int.coords:
    var_masked = var_df_int.where(~np.isnan(mask_all), drop = True)
elif 'xu_ocean' in var_df_int.coords:
    var_masked = var_df_int.where(~np.isnan(mask_vel), drop = True)

## Transforming masked data array into data frame
This will return values for all grid cells identified in the mask.

In [415]:
#Converting to pandas data frame
var_pd = var_masked.to_series().dropna().reset_index()

#Adding year and month column prior to merging with crabeater observations
var_pd['year'] = var_pd.apply(lambda i: i.time.year, axis = 1)
var_pd['month'] = var_pd.apply(lambda i: i.time.month, axis = 1)

#Finding name of columns to round up
round_cols = [i for i in var_pd.columns if 'ocean' in i]
#Rounding coordinate values prior to merging
var_pd = var_pd.round({round_cols[0]: 3, round_cols[1]: 3})
#Removing time column that is not needed
var_pd = var_pd.drop(columns = 'time')

#Checking results
var_pd.head()

,yt_ocean,xt_ocean,dist_km,year,month
0,-69.239,75.05,923.780931,1981,12
1,-69.239,75.85,928.570625,1981,12
2,-69.239,76.05,929.993389,1981,12
3,-69.239,77.25,936.971498,1981,12
4,-69.155,73.95,910.206697,1981,12


In [416]:
#Getting column names for merging
cols = var_pd.drop(columns = varDict['var_mod']).columns.tolist()
cols

['yt_ocean', 'xt_ocean', 'year', 'month']

## Joining masked data frame with background data frame
We will use the grid cell coordinates and dates to perform this join.

In [417]:
crabeaters = crabeaters.merge(var_pd, on = cols, how = 'left')
crabeaters.rename(columns = {varDict['var_mod']: varDict['var_short_name']}, inplace = True)
crabeaters

,date,year,month,yt_ocean,xt_ocean,yu_ocean,xu_ocean,season_year,life_stage,decade,...,SST_degC,bottom_temp_degC,SSS_PSU,bottom_sal_PSU,vel_lat_surf_msec,vel_lat_bottom_msec,vel_lon_surf_msec,vel_lon_bottom_msec,lt_pack_ice,dist_ice_edge_km
0,1999-12,1999,12,-69.239,75.05,-69.260,75.1,summer,weaning,1990,...,-1.687469,-1.868622,33.722919,34.600845,0.013854,-0.028669,-0.029011,-0.051852,0.595238,1022.477839
1,1999-12,1999,12,-69.239,75.85,-69.260,75.9,summer,weaning,1990,...,-1.335541,-1.865753,34.134918,34.597298,0.039097,0.058073,-0.099940,-0.030926,0.583333,1038.829585
2,1999-12,1999,12,-69.239,76.05,-69.260,76.1,summer,weaning,1990,...,-1.288300,-1.864441,34.247360,34.597195,-0.020978,0.006688,-0.126324,-0.078659,0.547619,1043.079696
3,1999-12,1999,12,-69.239,77.25,-69.260,77.3,summer,weaning,1990,...,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,0.000000,1065.696366
4,1999-12,1999,12,-69.155,73.95,-69.134,74.0,summer,weaning,1990,...,-1.397827,-1.866455,32.897396,34.570301,0.039995,-0.006844,-0.013034,0.001328,0.511905,993.399655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35561,1999-12,1999,12,-59.442,69.25,-59.468,69.3,summer,weaning,1990,...,-0.293945,-0.532471,33.669758,34.632214,-0.000384,-0.014342,0.054321,-0.005073,0.000000,81.449917
35562,1999-12,1999,12,-59.442,71.95,-59.468,72.0,summer,weaning,1990,...,-0.782349,-0.523865,33.660709,34.632725,0.016833,-0.000272,0.134342,0.009946,0.000000,33.766644
35563,1987-11,1987,11,-59.340,70.85,-59.366,70.9,autumn,weaning,1980,...,-1.013916,-0.528931,33.876839,34.632641,0.021629,0.003514,0.123648,0.000978,0.000000,104.505354
35564,1998-11,1998,11,-59.340,71.55,-59.366,71.6,autumn,weaning,1990,...,-0.833984,-0.528381,33.859493,34.632427,0.021677,-0.001377,0.062664,0.000898,0.000000,62.873381


## Saving data frame to disk

In [418]:
crabeaters.to_csv('../Cleaned_Data/unique_background_20x_obs_all_env.csv', index = False)

In [179]:
crabeaters = pd.read_csv('../Cleaned_Data/unique_background_20x_obs_all_env.csv')
crabeaters['date'] = crabeaters.apply(lambda x: f'{x.year}-{str(x.month).zfill(2)}', axis = 1)
crabeaters

,date,year,month,yt_ocean,xt_ocean,yu_ocean,xu_ocean,season_year,life_stage,decade,sector,zone,presence,bottom_slope_deg,dist_shelf_km,dist_coast_km,depth_m
0,1999-12,1999,12,-69.239,72.85,-69.260,72.9,summer,weaning,1990,East Indian,Antarctic,0,NaN,508.651552,144.923494,NaN
1,1987-11,1987,11,-69.155,75.25,-69.134,75.3,autumn,weaning,1980,Central Indian,Antarctic,0,89.695526,494.994640,131.113864,729.2120
2,1997-12,1997,12,-69.155,75.45,-69.134,75.5,summer,weaning,1990,Central Indian,Antarctic,0,89.907967,495.450580,131.866702,719.9167
3,1999-12,1999,12,-69.155,78.15,-69.134,78.2,summer,weaning,1990,East Indian,Antarctic,0,NaN,501.471683,181.594654,NaN
4,1999-12,1999,12,-69.070,71.05,-69.049,71.1,summer,weaning,1990,East Indian,Antarctic,0,NaN,487.710918,163.788036,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20112,1987-11,1987,11,-59.442,72.45,-59.468,72.5,autumn,weaning,1980,Central Indian,Antarctic,0,89.843651,536.622352,704.284410,4385.1665
20113,1999-12,1999,12,-59.340,70.75,-59.366,70.8,summer,weaning,1990,Central Indian,Antarctic,0,89.987152,552.156034,702.597893,4315.1807
20114,1999-12,1999,12,-59.340,71.85,-59.366,71.9,summer,weaning,1990,Central Indian,Antarctic,0,89.949501,548.045810,709.701986,4503.2637
20115,1984-11,1984,11,-59.340,72.65,-59.366,72.7,autumn,weaning,1980,Central Indian,Antarctic,0,89.961555,548.045810,717.588321,4334.3193
